In [23]:
#imports 
import torch as th
import numpy as np 
from torch import nn,optim
import torch.nn.functional as F

In [25]:
#making the dummy data 
inp = th.tensor([[0,1],[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,8],[8,9],[9.,10]], requires_grad= True)
target = th.tensor([[1],[2],[3],[4.],[5],[6],[7],[8],[9],[0]] , requires_grad= True)

#other dummy data
inp_2 = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad= True)
target_2 = th.tensor([[0],[0],[1],[1.]] , requires_grad= True)

In [26]:

#the toy model 
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.fc1 = nn.Linear(2,10)
        self.fc2 = nn.Linear(10,1)
    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

network = Network()
network

Network(
  (fc1): Linear(in_features=2, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)

In [28]:
#training method 
def train():
    #training logic 
    #train optimizer 
    opt = optim.SGD(params=network.parameters(), lr=0.1) 
    for i in range(20):
        #making the gradient zero
        opt.zero_grad()

        #make a prediction
        pred = network(inp_2)  #passing the data

        #calculation of the loss 
        loss = ((pred - target_2)**2).sum()

        #backward steps for backprop
        loss.backward()

        #stepping for optmizer 
        opt.step()

        #print the loss
        print(loss)

train()   

tensor(16.8271, grad_fn=<SumBackward0>)
tensor(7.9613, grad_fn=<SumBackward0>)
tensor(1.1399, grad_fn=<SumBackward0>)
tensor(0.9090, grad_fn=<SumBackward0>)
tensor(0.8528, grad_fn=<SumBackward0>)
tensor(0.8059, grad_fn=<SumBackward0>)
tensor(0.7629, grad_fn=<SumBackward0>)
tensor(0.7233, grad_fn=<SumBackward0>)
tensor(0.6868, grad_fn=<SumBackward0>)
tensor(0.6536, grad_fn=<SumBackward0>)
tensor(0.6238, grad_fn=<SumBackward0>)
tensor(0.6069, grad_fn=<SumBackward0>)
tensor(0.5767, grad_fn=<SumBackward0>)
tensor(0.4180, grad_fn=<SumBackward0>)
tensor(0.3380, grad_fn=<SumBackward0>)
tensor(0.2443, grad_fn=<SumBackward0>)
tensor(0.1805, grad_fn=<SumBackward0>)
tensor(0.1133, grad_fn=<SumBackward0>)
tensor(0.0896, grad_fn=<SumBackward0>)
tensor(0.0605, grad_fn=<SumBackward0>)


In [30]:
network(inp_2)

tensor([[-0.0364],
        [ 0.1046],
        [ 0.8309],
        [ 1.0566]], grad_fn=<AddmmBackward>)

In [32]:
#creating a hook 
import syft as sy
hook =sy.TorchHook(th)

In [33]:
#creation of the workers 
bob = sy.VirtualWorker(hook, id = "bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id= "alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker me already exists. Replacing old worker which could cause                     unexpected behavior


In [35]:
#encypting the model
encypted_model = network.fix_precision().share(alice, bob,crypto_provider= secure_worker)

In [39]:
#checking for the encypted model in the system 
list(encypted_model.parameters())

[Parameter containing:
 (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:36328489793 -> alice:6066174768]
 	-> [PointerTensor | me:49276056213 -> bob:51813811006]
 	*crypto provider: secure_worker*,
 Parameter containing:
 (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:35339522479 -> alice:27378373565]
 	-> [PointerTensor | me:32207285278 -> bob:55909070512]
 	*crypto provider: secure_worker*,
 Parameter containing:
 (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:46362332671 -> alice:11446719628]
 	-> [PointerTensor | me:73725396889 -> bob:88485665780]
 	*crypto provider: secure_worker*,
 Parameter containing:
 (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:9978673984 -> alice:1651963050]
 	-> [PointerTensor | me:74567219640 -> bob:50629375631]
 	*crypto provider: secure_worker*]

In [40]:
#encypting the data for input
encypted_data = inp_2.fix_precision().share(alice,bob, crypto_provider = secure_worker)

In [46]:
#ecncypting the data for the prediction
encypted_pred = inp_2.fix_precision().share(alice, bob, crypto_provider = secure_worker)

In [47]:
#prediction of the model
encypted_predition = encypted_model(encypted_data)
encypted_predition

/home/abhi/anaconda3/lib/python3.8/site-packages/syft/frameworks/torch/tensors/interpreters/additive_shared.py:122: UserWarning: Use dtype instead of field
  warnings.warn("Use dtype instead of field")


(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:23002103296 -> alice:34093406614]
	-> [PointerTensor | me:70269851014 -> bob:6467384377]
	*crypto provider: secure_worker*

In [48]:
encypted_predition.get().float_precision()

tensor([[-0.0350],
        [ 0.1050],
        [ 0.8310],
        [ 1.0550]])